In [1]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
product = Website("https://in.nothing.tech/pages/phone-2")
product.links

['/',
 'javascript:void(0);',
 '/pages/phone-2',
 '/pages/phone-2a',
 '/pages/phone-2a-plus',
 '/products/phone-2-case',
 '/products/phone-2-screen-protector',
 '/products/phone-2a-case',
 '/products/phone-2a-screen-protector',
 '/products/power-45w',
 '/products/cable',
 '/products/cable',
 'javascript:void(0);',
 '/products/ear-open',
 '/products/ear',
 '/products/ear-a',
 '/pages/audio',
 'https://in.cmf.tech',
 '/pages/store',
 'https://nothing.community/',
 '/pages/support-centre',
 '/account/login',
 '/pages/phone-2',
 '/pages/phone-2a',
 '/pages/phone-2a-plus',
 '/products/phone-2-case',
 '/products/phone-2-screen-protector',
 '/products/phone-2a-case',
 '/products/phone-2a-screen-protector',
 '/products/power-45w',
 '/products/cable',
 '/products/cable',
 '/products/ear-open',
 '/products/ear',
 '/products/ear-a',
 '/pages/audio',
 'https://in.cmf.tech',
 '/pages/store',
 'https://nothing.community/',
 '/pages/support-centre',
 '#pdp-overview',
 '#GLYPH-INTERFACE',
 '#nothingos

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the product, \
such as links to Specs page, or a support page, or reviews pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "specs page", "url": "https://full.url/goes/here/specs"},
        {"type": "highlights page": "url": "https://another.full.url/highlights"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the product, such as links to Specs page, or a support page, or reviews pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "specs page", "url": "https://full.url/goes/here/specs"},
        {"type": "highlights page": "url": "https://another.full.url/highlights"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the product, respond with the full https URL in JSON format. \
        Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(product))

Here is the list of links on the website of https://in.nothing.tech/pages/phone-2 - please decide which of these are relevant web links for a brochure about the product, respond with the full https URL in JSON format.         Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
/
javascript:void(0);
/pages/phone-2
/pages/phone-2a
/pages/phone-2a-plus
/products/phone-2-case
/products/phone-2-screen-protector
/products/phone-2a-case
/products/phone-2a-screen-protector
/products/power-45w
/products/cable
/products/cable
javascript:void(0);
/products/ear-open
/products/ear
/products/ear-a
/pages/audio
https://in.cmf.tech
/pages/store
https://nothing.community/
/pages/support-centre
/account/login
/pages/phone-2
/pages/phone-2a
/pages/phone-2a-plus
/products/phone-2-case
/products/phone-2-screen-protector
/products/phone-2a-case
/products/phone-2a-screen-protector
/products/power-45w
/products/cable
/products/cable
/products/ear-open
/products/ear
/pr

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
get_links("https://in.nothing.tech/pages/phone-2")

{'links': [{'type': 'specs page',
   'url': 'https://in.nothing.tech/pages/phone-2'},
  {'type': 'support page',
   'url': 'https://in.nothing.tech/pages/support-centre'},
  {'type': 'phone case product page',
   'url': 'https://in.nothing.tech/products/phone-2-case'},
  {'type': 'screen protector product page',
   'url': 'https://in.nothing.tech/products/phone-2-screen-protector'},
  {'type': 'phone accessories page',
   'url': 'https://in.nothing.tech/products/cable'},
  {'type': 'power product page',
   'url': 'https://in.nothing.tech/products/power-45w'},
  {'type': 'ear product page',
   'url': 'https://in.nothing.tech/products/ear-open'}]}

In [13]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    # print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
        result = result[:8_000]
    return result

In [14]:
print(get_all_details("https://in.nothing.tech/pages/phone-2"))

Landing page:
Webpage Title:
Nothing Phone (2) - Nothing India
Webpage Contents:
NOTHING (R)
Phone
Phones
Phone (2)
Phone (2a)
Phone (2a) Plus
NEW
Accessories
Phone (2) case
Phone (2) Screen Protector
Phone (2a) / Plus Case
Phone (2a) / Plus Screen Protector
Power (45 W)
Cable (c-c) 100 cm
Cable (c-c) 180 cm
Audio
Earbuds
Ear (open)
NEW
Ear
Ear (a)
View all
CMF
Store
Community
Support
Phone
Phones
Phone (2)
Phone (2a)
Phone (2a) Plus
NEW
Accessories
Phone (2) case
Phone (2) Screen Protector
Phone (2a) / Plus Case
Phone (2a) / Plus Screen Protector
Power (45 W)
Cable (c-c) 100 cm
Cable (c-c) 180 cm
Audio
Earbuds
Ear (open)
NEW
Ear
Ear (a)
View all
CMF
Store
Community
Support
Phone (2)
Glyph Interface
Nothing OS 3.0
Using Nothing OS 3.0
Camera
Premium
Sustainability
Specs
Compare
CLOSE
phone (2)
<
Spin: 360
>
<
Spin: 360
>
0
of
0
"Nothing has melded significant refinements within a grander scheme of things"
Hindustan Times
"iPhone of Android World Back with ‘Lighting’ Superpowers"
Dainik

In [15]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a product website \
and creates a short brochure about the product for prospective customers, investors and recruits. Respond in markdown.\
Include details of product specs, highlightes , reviews and some more relavent ddata, if you have the information."

In [17]:
def get_brochure_user_prompt(product_name, url):
    user_prompt = f"You are looking at a product called: {product_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the product in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:8_000] # Truncate if more than 5,000 characters
    return user_prompt

In [ ]:
get_brochure_user_prompt("Nothing Phone (2)", "https://in.nothing.tech/pages/phone-2")

In [18]:
def create_brochure(product_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(product_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [19]:
create_brochure("Nothing Phone (2)", "https://in.nothing.tech/pages/phone-2")

# **Nothing Phone (2)**

## **Welcome to the Future of Smartphone Design and Functionality**

**The Nothing Phone (2)** is here to revolutionize your mobile experience with cutting-edge technology, unique design, and unparalleled user-friendliness. Here’s a quick overview of what makes this smartphone a game-changer.

---

### **Key Features**

- **Display**: 
  - **6.7" LTPO AMOLED** - A stunning flexible display with vibrant colors and deep contrasts.

- **Camera**: 
  - **Dual Rear Camera**: 50 MP main and 50 MP ultra-wide camera with AI enhancements for incredible photos.
  - **Front Camera**: 32 MP sensor - Capture every detail in stunning clarity.
  - **Advanced HDR**: Provides dynamic image accuracy with significantly improved noise reduction.

- **Processor**: 
  - **Snapdragon® 8+ Gen 1** - Experience an **80% boost in performance** over its predecessor.

- **Battery**: 
  - Fast charging capability with **45W power** and exceptional energy efficiency.

- **Operating System**: 
  - Runs on **Nothing OS 3.0** - Customized experience with functional personalization and 3 years of major updates.

- **Glyph Interface**: 
  - Innovative, customizable light notifications - Assign different alerts to different contacts and keep connected without distraction.

---

### **Variations**

Available in multiple configurations:
- **Dark Grey**
  - 8GB RAM + 128GB Storage
  - 12GB RAM + 256GB Storage
  - 12GB RAM + 512GB Storage
- **White**
  - 12GB RAM + 256GB Storage
  - 12GB RAM + 512GB Storage

---

### **Sustainability Commitment**

The Nothing Phone (2) combines premium engineering with a commitment to sustainability, minimizing environmental impact while maximizing user experience.

---

### **Reviews**

> "Nothing has melded significant refinements within a grander scheme of things."  
> — *Hindustan Times*

> "iPhone of Android World Back with ‘Lighting’ Superpowers."  
> — *Dainik Jagran*

> "A unique, powerful phone which puts user experience at the forefront."  
> — *My Smart Price*

> "Myriad hues of a monochrome marvel."  
> — *The Hindu Business Line*

---

### **Why Choose Nothing Phone (2)?**

- **Enhanced User Experience**: The Glyph Interface and Nothing OS 3.0 enhance how you interact with your phone, making communication seamless.
- **Superior Photography**: With advanced algorithms and a top-tier camera, capture every moment beautifully.
- **Power Meets Performance**: Enjoy lightning-fast speeds and an efficient experience powered by state-of-the-art technology.
- **Stylish and Functional Design**: A blend of elegance and practicality ensures this phone stands out in any setting.

---

### **Join the Nothing Community**

Become a part of an innovative community that prioritizes real connections, sustainable practices, and pioneering technology. Share your experiences with #withNothing on social platforms and get featured!

---

### **Experience Nothing Like Ever Before**

Don’t just take our word for it; experience the superior design, performance, and functionality for yourself. The **Nothing Phone (2)** is your next essential gadget.

**[Buy Now](#)**  

--- 

Your journey to unprecedented mobile experience begins here. Welcome to Nothing.